##### Домашнее задание

## Урок №6 - Оконные функции

#### Работу выполнил: Зензеров Никита Евгеньевич

### Для периода с 01.08.2018 по 07.08.2018 для каждой даты вывести саму дату, ID персонажа, имеющего наибольшую сумму транзакций за эту дату, и саму сумму транзакций по таблице shop_log. Если персонажи имеют одинаковую сумму транзакций, то они все должны попасть в конечную выборку. Данные отсортировать по датам по возрастанию. Для этого исходя из задания необходимо выбрать функцию ранжирования, для каждой даты присвоив персонажу его место исходя из суммы транзакций. То есть сгруппировать внутри оконной функции данные по датам и отсортировать по сумме транзакций по убыванию. Затем необходимо оставить только записи, относящиеся к первым местам.

In [2]:
date_max_transaction = pd.read_sql(f'''
SELECT time, charid, sum_transaction
FROM(
    SELECT CAST(time as date) AS time, charid, SUM(itemprice * itemcount) AS sum_transaction,
        DENSE_RANK() OVER(PARTITION BY CAST(time as date) 
                          ORDER BY SUM(itemprice * itemcount) DESC) AS dr
    FROM {GROUP_SCHEMA}.shop_log
    WHERE CAST(time as date) BETWEEN '2018-08-01' AND '2018-08-07'
    GROUP BY 1, 2) AS t
WHERE dr = 1    
''', con=engine)
date_max_transaction

,time,charid,sum_transaction
0,2018-08-01,3893933,6700.0
1,2018-08-02,5795259,5200.0
2,2018-08-02,4943833,5200.0
3,2018-08-03,4855383,11000.0
4,2018-08-04,5730766,10000.0
5,2018-08-05,2913253,5200.0
6,2018-08-06,5709710,7200.0
7,2018-08-07,5745069,6950.0


### Для периода с 01.08.2018 по 07.08.2018 вывести дату, сумму транзакций за эту дату и разницу суммы транзакцией и суммой предыдущего дня по таблице shop_log. Данные отсортировать по датам по возрастанию. Для этого необходимо использовать функцию LAG(), с её помощью вывести отдельным полем сумму транзакций предыдущего дня, и просто вычесть одно из другого.

In [11]:
transaction_diff = pd.read_sql(f'''
SELECT CAST(time AS date) AS time, SUM(itemprice * itemcount) AS sum_transaction,
    CAST(SUM(itemprice * itemcount) - LAG(SUM(itemprice * itemcount), 1, SUM(itemprice * itemcount) 
    ) OVER () AS int) AS transaction_day_diff
FROM {GROUP_SCHEMA}.shop_log
WHERE CAST(time as date) BETWEEN '2018-08-01' AND '2018-08-07'
GROUP BY 1
ORDER BY 1
''', con=engine)
transaction_diff

,time,sum_transaction,transaction_day_diff
0,2018-08-01,304900.0,0
1,2018-08-02,272000.0,-32900
2,2018-08-03,327150.0,55150
3,2018-08-04,311050.0,-16100
4,2018-08-05,298900.0,-12150
5,2018-08-06,367250.0,68350
6,2018-08-07,308350.0,-58900
